In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
# Import 
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

import plotly.express as px
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

import warnings
warnings.filterwarnings("ignore")

## Tasks:
1) Which store has maximum sales

2) Which store has maximum standard deviation i.e., the sales vary a lot. Also, find out the coefficient of mean to standard deviation

3) Which store/s has good quarterly growth rate in Q3’2012

4) Some holidays have a negative impact on sales. Find out holidays which have higher sales than the mean sales in non-holiday season for all stores together

5) Provide a monthly and semester view of sales in units and give insights

In [ ]:
# Readiin in data in dataframe
wm = pd.read_csv('/kaggle/input/retail-analysis-with-walmart-sales-data/WALMART_SALES_DATA.csv')
wm.head()

In [ ]:
# Basic check up for missing value
wm.isna().sum()

# Task 1 - Which store has maximum sales

In [ ]:
wm.groupby('Store').sum()['Weekly_Sales'].sort_values(ascending = False).head()

**Task 1 Answer** - Store 20 had the highest sales with 3.013978e+08 dollar

# Task 2 - Which store has maximum standard deviation i.e., the sales vary a lot. Also, find out the coefficient of mean to standard deviation

In [ ]:
wm.groupby('Store').std()['Weekly_Sales'].sort_values(ascending = False).head()

In [ ]:
# Calculating Coefficient of Variation (CV)

# Equation is CV = The Standard Deviation of dataset / The mean of dataset

cv = wm.groupby('Store').std()['Weekly_Sales'] / wm.groupby('Store').mean()['Weekly_Sales']
cv = cv.reset_index().rename(columns = {'Weekly_Sales': 'Coefficient of Variation'})

cv.head()

In [ ]:
# Maximum CV
cv.sort_values(by='Coefficient of Variation', ascending = False).head()

**Task 2 Anwer** - Store 14 had the maximum standard deviation in Weekly Sales. Additionally, Store 35 has the maximum coefficient of variation at approximately 0.299

# Task 3 - Which store/s has good quarterly growth rate in Q3’2012

In [ ]:
# Convert Date column to datetime object
wm['Date'] = pd.to_datetime(wm['Date'], format="%d-%m-%Y")
wm.info()

In [ ]:
# Extract the year and month
wm['Year'] = pd.DatetimeIndex(wm['Date']).year
wm['Month'] = pd.DatetimeIndex(wm['Date']).month
wm.head()

In [ ]:
# Quarter Three is from month July (6) to September (9) and Year 2012

wm_q3_2012 = wm[(wm['Month'].isin([6,7,8,9])) & (wm['Year'] == 2012)] 
wm_q3_2012.head()

In [ ]:
fig = px.bar(data_frame = wm_q3_2012.groupby('Store').sum().reset_index(),
             x = 'Store', y = 'Weekly_Sales', text = 'Weekly_Sales')


fig.update_layout(title = 'Total Weekly Sales of 45 Walmart stores during Q3 of 2012',
                  yaxis_title = 'Total Weekly Sales',
                  font = dict(family = "Courier New, monospace",
                              size = 14, color = 'black')
                  )

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [n for n in range(1,46)],
    )
)



fig.show()

### Percentage Change

In [ ]:
# Growth Rate by Store so first recorded date of quarter 3 to last date
wm_q3_2012['Date'].iloc[0] , wm_q3_2012['Date'].iloc[-1]

In [ ]:
gr_wm = wm_q3_2012[(wm_q3_2012['Date'] == '2012-06-01') | (wm_q3_2012['Date'] == '2012-09-28')]
pct_wm = gr_wm.groupby('Store')['Weekly_Sales'].pct_change().dropna().reset_index().rename(columns={'index':'Store','Weekly_Sales':'%Change'})
pct_wm['Store'] = gr_wm['Store'].unique()
pct_wm.head()

In [ ]:
# Top Performing WM Stores during Q3 2012
pct_wm.sort_values(by='%Change',ascending=False).head()

In [ ]:
# Decrease in weekly sales a lot during Q3 2012
pct_wm.sort_values(by='%Change',ascending=False).tail()

**Task 3 Answer** - Store 17 and 44 were the only two Wm Stores that have increased their weekly sales during Quarter 3 of year 2012. Other than those two, other stores had decreased their weekly sales. Although Store 4 had the highest weekly sales during this time period, their sales actually went down.

# Task 4 - Some holidays have a negative impact on sales. Find out holidays which have higher sales than the mean sales in non-holiday season for all stores together

Note on Holiday Events -->
* Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
* Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
* Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
* Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

In [ ]:
# Creating Holiday DataFrame
holiday = wm[wm['Holiday_Flag'] == 1]
holiday.tail()

In [ ]:
# What are holiday dates present here?
holiday['Date'].value_counts()

In [ ]:
# Assigning Holiday Name to Each presented date to make it easier for the comparison 

# Super Bowl: 12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13
# Labour Day: 10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13
# Thanksgiving: 26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13
# Christmas: 31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13

from datetime import datetime

super_bowl = [datetime.strptime(date,"%d-%b-%y").date() for date in '12-Feb-10, 11-Feb-11, 10-Feb-12, 8-Feb-13'.split(", ")]
labour_day = [datetime.strptime(date,"%d-%b-%y").date() for date in '10-Sep-10, 9-Sep-11, 7-Sep-12, 6-Sep-13'.split(", ")]
thanksgiving = [datetime.strptime(date,"%d-%b-%y").date() for date in '26-Nov-10, 25-Nov-11, 23-Nov-12, 29-Nov-13'.split(", ")]
christmas = [datetime.strptime(date,"%d-%b-%y").date() for date in '31-Dec-10, 30-Dec-11, 28-Dec-12, 27-Dec-13'.split(", ")]

def assign_holiday(date):
    if date in super_bowl:
        return 'Super Bowl'
    elif date in labour_day:
        return 'Labor Day'
    elif date in thanksgiving:
        return 'Thanksgiving'
    elif date in christmas:
        return 'Christmas'
    else:
        return 'Not Holiday'
    
holiday['Occasion'] = holiday['Date'].apply(lambda date: assign_holiday(date))
holiday.head()

In [ ]:
holiday_year = holiday.groupby(['Year','Occasion']).sum().reset_index()

fig = px.bar(data_frame = holiday_year, 
             x = 'Year', y = 'Weekly_Sales',
             color = 'Occasion', barmode = 'group',
             text = 'Weekly_Sales', height = 550,
             color_discrete_sequence = px.colors.qualitative.Safe)

fig.update_layout(title = 'Walmart Total Sales from 2010 to 2012 by Public Holiday',
                  yaxis_title = 'Total Sales',
                  legend_title = 'Holiday',
                  font = dict(family = "Courier New, monospace",
                              size = 14, color = 'black')
                  )

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.update_layout(
    xaxis = dict(
        tickmode = 'array',
        tickvals = [n for n in range(2010,2013)],
    )
)

fig.show()

In [ ]:
# Mean sales in non-holiday season for all stores together
non_holi_mean_sales = wm[wm['Holiday_Flag'] == 0]['Weekly_Sales'].mean()
non_holi_mean_sales / 10**6

In [ ]:
# Holiday Sales that is greater than mean 
holiday.groupby('Occasion')['Weekly_Sales'].mean() / 10**6 # Unit in Million (easier for comparison)

# Task 5 - Provide a monthly and semester view of sales in units and give insights

In [ ]:
# Top 3 Total Sales Walmart across 2 year
fig = px.line(data_frame = wm[wm['Store'].isin(wm.groupby('Store').sum().sort_values(by='Weekly_Sales',ascending = False).iloc[:3].index.to_list())],
              x = 'Date', y = 'Weekly_Sales',
              color = 'Store', color_discrete_sequence = px.colors.qualitative.Safe)

fig.update_layout(title = 'Top 3 Walmart Stores (by Total Sales) Weekly Sales',
                  yaxis_title = 'Weekly Sales',
                  font = dict(family = "Courier New, monospace",
                              size = 14, color = 'black')
                  )

fig.show()

In [ ]:
# Monthly Sales
import calendar

fig = px.bar(data_frame = wm.groupby('Month').sum().reset_index(),
             x = 'Month', y = 'Weekly_Sales',
             text = 'Weekly_Sales', height = 550)

fig.update_layout(title = 'Walmart Overall Monthly Sales from 2011 to 2013',
                  yaxis_title = 'Total Sales',
                  font = dict(family = "Courier New, monospace",
                              size = 14, color = 'black')
                  )

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.update_yaxes(tickprefix="$")

fig.update_layout(
    xaxis = dict(
        ticktext = [calendar.month_name[n] for n in range(1,13)],
        tickvals = [n for n in range(1,13)]
    )
)

fig.show()

In [ ]:
fig = px.bar(data_frame = wm.groupby(['Month','Year']).sum().reset_index(),
             x = 'Month', y = 'Weekly_Sales', color = 'Year',
             text = 'Weekly_Sales', height = 550)

fig.update_layout(title = 'Walmart Monthly Sales by Year',
                  yaxis_title = 'Total Sales',
                  font = dict(family = "Courier New, monospace",
                              size = 14, color = 'black')
                  )

fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')

fig.update(layout_coloraxis_showscale=False)

fig.update_yaxes(tickprefix="$")

fig.update_layout(
    xaxis = dict(
        ticktext = [calendar.month_name[n] for n in range(1,13)],
        tickvals = [n for n in range(1,13)]
    )
)

fig.show()

In [ ]:
# By Year Sales
plt.figure(dpi=120)
sns.barplot(data = wm.groupby('Year').sum().reset_index(),
            x = 'Year', y = 'Weekly_Sales', palette = 'Set2')
plt.title("Yearly Sales")
plt.ylabel("Sales (dollar)")
plt.show()

# Statistical Model
**Task -** For Store 1 – Build prediction models to forecast demand (Linear Regression – Utilize variables like date and restructure dates as 1 for 5 Feb 2010 (starting from the earliest date in order). Hypothesize if CPI, unemployment, and fuel price have any impact on sales.) Change dates into days by creating new variable.
Select the model which gives best accuracy.

In [ ]:
# Recap of how does the Data Look like
wm.head()

In [ ]:
# Adding More columns
wm['Day'] = pd.DatetimeIndex(wm['Date']).day
wm['Holiday'] = wm['Date'].apply(lambda date: assign_holiday(date))


wm.head()

In [ ]:
# Checking for outlier and NaN value

features_list = 'Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day'.split(", ")

plt.figure(dpi=150)
count = 1
for feature in features_list:
    plt.subplot(4,2,count)
    sns.boxplot(wm[feature])
    count += 1
plt.tight_layout()
plt.show()

In [ ]:
# Removing Outlier

def remove_out(feature):

    p25 = wm[feature].quantile(0.25)
    p75 = wm[feature].quantile(0.75)
    iqr = p75 - p25
    
    upper_limit = p75 + 1.5 * iqr 
    lower_limit = p25 - 1.5 * iqr
    
    new_df = wm[(wm[feature] > lower_limit) & (wm[feature] < upper_limit)]
    
    return new_df

for feature in features_list:
    wm = remove_out(feature)
wm.shape

In [ ]:
from sklearn.preprocessing import OrdinalEncoder

ordinal_encoder = OrdinalEncoder()
wm['Holiday'] = ordinal_encoder.fit_transform(wm[['Holiday']])

print(ordinal_encoder.categories_)

In [ ]:
wm.head()

In [ ]:
corr_matrix = wm.corr()
corr_matrix['Weekly_Sales'].sort_values(ascending = False)

In [ ]:
from sklearn.model_selection import train_test_split

features = 'Temperature, Fuel_Price, CPI, Unemployment, Year, Month, Day, Holiday'.split(", ")
target = 'Weekly_Sales'

X = wm[features]
y = wm[target]

X_train, X_test, y_train, y_test = train_test_split(X,y,test_size = 0.2)

In [ ]:
from sklearn.linear_model import LinearRegression

lin_reg = LinearRegression()
lin_reg.fit(X_train, y_train)

prediction = lin_reg.predict(X_test)

In [ ]:
from sklearn.metrics import mean_squared_error

lin_rmse = np.sqrt(mean_squared_error(y_test, prediction))
print("RSME:", lin_rmse)
print("Score:", lin_reg.score(X_train, y_train) * 100,"%")

In [ ]:
sns.scatterplot(prediction, y_test)

In [ ]:
from sklearn.tree import DecisionTreeRegressor

tree_reg = DecisionTreeRegressor()
tree_reg.fit(X_train, y_train)

tree_prediction = tree_reg.predict(X_test)
tree_rmse = np.sqrt(mean_squared_error(y_test, tree_prediction))
print("RMSE:",tree_rmse)
print("Score:", tree_reg.score(X_train, y_train) * 100, "%")

In [ ]:
sns.scatterplot(tree_prediction, y_test)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
forest_reg = RandomForestRegressor()
forest_reg.fit(X_train, y_train)

forest_prediction = forest_reg.predict(X_test)
forest_rmse = np.sqrt(mean_squared_error(y_test, forest_prediction))
print("RMSE:",forest_rmse)
print("Score:", forest_reg.score(X_train, y_train) * 100, "%")

In [ ]:
sns.scatterplot(forest_prediction, y_test)

In [ ]:
# Fine-Tune model using GridSearch
from sklearn.model_selection import GridSearchCV

param_grid = [
    {'n_estimators': [3, 10, 30, 45, 60], 'max_features': [2,4,6,8]},
]

forest_reg = RandomForestRegressor()
grid_search = GridSearchCV(forest_reg, param_grid, cv = 5,
                           scoring = 'neg_mean_squared_error',
                           return_train_score = True)

grid_search.fit(X_train, y_train)

In [ ]:
grid_search.best_params_

# Thank you for checking out my work!